# MNIST

### MNIST 데이터 받기

In [1]:
import os
import sys
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("data/", one_hot=True)
train_data = mnist.train.images
train_label = mnist.train.labels
test_data = mnist.test.images
test_label = mnist.test.labels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


### MNIST 데이터 확인하기

In [2]:
# size of MNIST
print(train_data.shape)
print(train_label.shape)
print(test_data.shape)
print(test_label.shape)

(55000, 784)
(55000, 10)
(10000, 784)
(10000, 10)


### Data Pre-processing (데이터 전처리)

# Implementation (1)


## Loss function (손실 함수) : Cross Entropy

# <center> \\( L(y_i, f(x_i; W)) = -\frac{1}{n}\sum_{i=1}^{n}\sum_{k=1} y_{i,j} log(f(x_i)_k)\\)</center>


#### get_cross_entropy_loss 함수의 내용을 완성하세요.
#### (Hint : (1) tf.reduce_mean(), tf.reduce_sum(), tf.log() (2) Tensor dimension에 유의 (3) log 함수 사용 시 epsilon 사용하세요.)

In [3]:
def get_cross_entropy_loss(y_true, y_hat, epsilon=1e-8):
    """
    compute cross entropy

    Args:
        y_true: true label
        y_hat: predicted label
        epsilon: small value to prevent NaN in log

    Returns:
        cross entropy loss
    """
    with tf.name_scope('cross_entropy'):
        loss = -tf.reduce_mean(tf.reduce_sum((y_true * tf.log(y_hat)), axis = 1))
        loss = -tf.reduce_mean(tf.reduce_sum(y_true * tf.log(y_hat), axis = 1))
    return loss

y_true = tf.placeholder(tf.float32, [None, 10])
y_hat = tf.placeholder(tf.float32, [None, 10])
get_cross_entropy_loss(y_true, y_hat)


def get_accuracy(y_true, y_hat):
    """
    compute cross entropy

    Args:
        y_true: true label
        y_hat: predicted label
        epsilon: small value to prevent NaN in log

    Returns:
        cross entropy loss
    """
    with tf.name_scope('accuracy'):
        # Compare the highest indices between the predicted label and the true label
        correct_prediction = tf.equal(tf.argmax(y_hat, 1), tf.argmax(y_true, 1), name='correct_prediction')
        # Compute accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
    return accuracy

## Hyper-parameter (하이퍼 파라미터)

In [8]:
# Set hypyerparameters
learning_rate = 0.001
max_iter = 2000
batch_size = 100

# Implementation (2)
## Linear Classifier (선형 분류기)

## <center> \\( f(x) = W^Tx+b \\)</center>

### linear function을 완성하세요.
### (Hint : (1) weight, bias 선언 (2) tf.get_variable()의 initializer  (3) tf.matmul())

In [9]:
def fc(name, out_dim, inputs):
    """
    Args :
        Inputs : Input tensor
        out_dim : output dimension
        
    Returns:
        inputs * weight + bias
    """
    with tf.variable_scope(name):
        shp = inputs.get_shape().as_list()[-1]
        initials = tf.truncated_normal([shp, out_dim], stddev = 0.01)
        weight = tf.get_variable('w', initializer = initials)

        initials = tf.constant(1.0 , shape = [out_dim])
        bias = tf.get_variable('b', initializer = initials)
        result = tf.matmul(inputs, weight) + bias
        return result        



# Implementation (3)

## Model Setting

### 1. Training data 및 Test data의 각각의 image를 한 vector로 만들어서 train_data, test_data에 각각 저장하세요.
#### Hint) 데이터 차원.
### 2. Dataset로부터 받은 데이터(Image, label)를 담을 변수를 각각 x 및 y_true에 선언하세요.
#### Hint) tf.placeholder
### 3. Implementation (2)에서 구현한 linear classifier 함수값과 softmax 함수를 통한 prediction 값을 y_hat에 저장하세요.
#### Hint) tf.nn.softmax
### 4. 3으로부터 얻은 결과를 통해 Implementation (1)에서 구현한 loss function을 통해 얻은 loss를 cross_entropy에 저장하세요.

In [10]:
tf.reset_default_graph()
# Flatten data
#train_data = None
#test_data = None

###################################################################
#                    Implementation 3-2                           #
###################################################################
x = tf.placeholder(tf.float32, [None, 784])
y_true = tf.placeholder(tf.float32, [None, 10])

###################################################################
#                    Implementation 3-3                          #
###################################################################

h1 = fc('layer1', 512, x)
h1 = tf.nn.relu(h1)
h2 = fc('layer2', 64, h1)
h2 = tf.nn.relu(h2)
y_logits = fc('layer3', 10, h2)
y_hat = tf.nn.softmax(y_logits)

###################################################################
#                    Implementation 3-4                           #
###################################################################
cross_entropy = get_cross_entropy_loss(y_true, y_hat)


# Calculate accuracy
accuracy = get_accuracy(y_true, y_hat)
# Make gradient descent op
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

# Make op to initialize declared variable
init = tf.global_variables_initializer()


with tf.Session() as sess:
    # Initialize variables
    sess.run(init)
    
    # Training
    for step in range(max_iter):
        # Get batch data and label
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # train the network and calculate cross entropy
        _ = sess.run(train_step, feed_dict={x: batch_x, y_true: batch_y})
        loss = sess.run(cross_entropy, feed_dict={x: batch_x, y_true: batch_y})
 
        # calcualte accuracy
        acc = sess.run(accuracy, feed_dict={x: test_data, y_true: test_label})
        # print loss (cross entropy) and accuracy at every 10th step
        if (step + 1) % 10 == 0:
            print("{}th iteration, loss: {:.4f}, test accuracy: {:.4f}".format(step + 1, loss, acc))

10th iteration, loss: 2.2592, test accuracy: 0.3236
20th iteration, loss: 2.0361, test accuracy: 0.4624
30th iteration, loss: 1.2936, test accuracy: 0.5701
40th iteration, loss: 0.9738, test accuracy: 0.7049
50th iteration, loss: 0.9719, test accuracy: 0.7329
60th iteration, loss: 0.8320, test accuracy: 0.7713
70th iteration, loss: 0.5047, test accuracy: 0.7972
80th iteration, loss: 0.5871, test accuracy: 0.8124
90th iteration, loss: 0.4481, test accuracy: 0.8444
100th iteration, loss: 0.4562, test accuracy: 0.8634
110th iteration, loss: 0.3058, test accuracy: 0.8690
120th iteration, loss: 0.4986, test accuracy: 0.8707
130th iteration, loss: 0.4111, test accuracy: 0.8794
140th iteration, loss: 0.5692, test accuracy: 0.8803
150th iteration, loss: 0.4317, test accuracy: 0.8894
160th iteration, loss: 0.3024, test accuracy: 0.8900
170th iteration, loss: 0.4848, test accuracy: 0.8896
180th iteration, loss: 0.3325, test accuracy: 0.8963
190th iteration, loss: 0.3316, test accuracy: 0.8997
20

1550th iteration, loss: 0.2037, test accuracy: 0.9686
1560th iteration, loss: 0.0292, test accuracy: 0.9676
1570th iteration, loss: 0.0793, test accuracy: 0.9687
1580th iteration, loss: 0.0746, test accuracy: 0.9691
1590th iteration, loss: 0.0509, test accuracy: 0.9681
1600th iteration, loss: 0.0522, test accuracy: 0.9673
1610th iteration, loss: 0.0995, test accuracy: 0.9678
1620th iteration, loss: 0.1204, test accuracy: 0.9697
1630th iteration, loss: 0.0633, test accuracy: 0.9650
1640th iteration, loss: 0.1237, test accuracy: 0.9687
1650th iteration, loss: 0.0517, test accuracy: 0.9661
1660th iteration, loss: 0.0536, test accuracy: 0.9654
1670th iteration, loss: 0.0639, test accuracy: 0.9663
1680th iteration, loss: 0.1041, test accuracy: 0.9669
1690th iteration, loss: 0.1326, test accuracy: 0.9668
1700th iteration, loss: 0.1155, test accuracy: 0.9701
1710th iteration, loss: 0.0988, test accuracy: 0.9684
1720th iteration, loss: 0.0547, test accuracy: 0.9690
1730th iteration, loss: 0.10

In [11]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1314668810350246369, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9928354404972487676
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9517196740019699449
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 7691812864
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12209395223807037889
 physical_device_desc: "device: 0, name: GeForce RTX 2080, pci bus id: 0000:17:00.0, compute capability: 7.5", name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 7251204506
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10447876293122012690
 physical_device_desc: "device: 1, name: GeForce RTX 2080, pci bus id: 0000:b3:00.0, compute capability: 7.5"]